# Finding the coarse correlated equilibrium (CCE) using Python

Here is an example of how we can implement an algorithm to find the coarse correlated equilibrium (CCE) in a 2-player game using Python.

# What is coarse correlated equilibrium?

Coarse correlated equilibrium (CCE) is a concept in game theory, specifically in the field of repeated games with incomplete information. It extends the idea of correlated equilibrium to repeated games.

In game theory, an equilibrium is a state where no player has an incentive to deviate from their current strategy given the strategies of the other players. In a correlated equilibrium, a third party (a mediator) suggests a probability distribution over the joint strategies of the players. Each player then independently chooses their strategy based on the advice received from the mediator.

A coarse correlated equilibrium relaxes the requirement of a precise correlation device. In a CCE, players' strategies are to commit to the correlation device or to deviate and play a different strategy. A coarse correlated equilibrium exists when accepting the device is the best response for each player when all others are also accepting.

The concept is particularly relevant in settings where players may not have access to a centralized mediator or communication channel but can use imprecise signals or randomization to achieve coordination.

# Defining the algorithm in Python

First, we want to define a function "cce", which takes as input a tuple "game" representing the payoffs for the two players as well as two optional parameters "num_samples" and "tolerance". 

"num_samples" determines the number of samples to take when approximating the CCE, and tolerance controls the level of tolerance for convergence of the algorithm.

In [3]:
import numpy as np

def cce(game, num_samples=1000, tolerance=1e-6):
    # game is a tuple of two matrices representing the payoffs for players 1 and 2
    num_strategies_p1 = game[0].shape[1]
    num_strategies_p2 = game[1].shape[1]
    strategy_probs = np.ones((num_strategies_p1, num_strategies_p2)) / (num_strategies_p1 * num_strategies_p2)

    for _ in range(num_samples):
        # Sample a strategy profile from the distribution
        p1_strategy, p2_strategy = np.unravel_index(np.random.choice(strategy_probs.size, p=strategy_probs.ravel()), strategy_probs.shape)
        
        # Calculate the expected payoff for each player at this strategy profile
        expected_payoff_p1 = np.sum(game[0][:,p2_strategy] * strategy_probs[p1_strategy,p2_strategy])
        expected_payoff_p2 = np.sum(game[1][p1_strategy,:] * strategy_probs[p1_strategy,p2_strategy])
        
        # Update the strategy probability distribution
        strategy_probs[p1_strategy,p2_strategy] *= (1 + tolerance * (expected_payoff_p1 - expected_payoff_p2))
        strategy_probs /= np.sum(strategy_probs)
    
    # The final strategy probability distribution is a CCE
    return strategy_probs


The algorithm works by iteratively sampling a strategy profile from the current probability distribution over strategies, and updating the distribution based on the expected payoffs at that profile. The process continues until the probability distribution converges to a fixed point.

To run the function, simply call it with the game matrix as input, like this:

In [4]:
game = (np.array([[3, 0], [5, 1]]), np.array([[3, 5], [0, 1]]))
strategy_probs = cce(game)
print(strategy_probs)


[[0.24999877 0.24989338]
 [0.25010907 0.24999877]]


This would output the probability distribution over strategies that represents a CCE for the given game.